# Time series analysis

In this notebook, we will learn how to use `rocky-worlds-utils` to generate spectral time-series and calculate a light curve from STIS and COS data.

## Imports

- `astropy`: FITS files I/O
- `matplotlib`: Plotting library
- `numpy`: array calculations
- `rocky_worlds_utils`: various functions to facilitate data analysis

In [2]:
%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from rocky_worlds_utils.hst import time_series, stis_analysis

pylab.rcParams['figure.figsize'] = 15.0,9.0
pylab.rcParams['font.size'] = 18

/Users/ldsantos/Applications/miniconda3/envs/stenv/lib/python3.12/site-packages/stsci/tools/nmpfit.py:8: UserWarning: NMPFIT is deprecated - stsci.tools v 3.5 is the last version to contain it.
  warnings.warn("NMPFIT is deprecated - stsci.tools v 3.5 is the last version to contain it.")
/Users/ldsantos/Applications/miniconda3/envs/stenv/lib/python3.12/site-packages/stsci/tools/gfit.py:18: UserWarning: GFIT is deprecated - stsci.tools v 3.4.12 is the last version to contain it.Use astropy.modeling instead.
  warnings.warn("GFIT is deprecated - stsci.tools v 3.4.12 is the last version to contain it."


The following tasks in the stistools package can be run with TEAL:
   basic2d      calstis     ocrreject     wavecal        x1d          x2d


In the notebook \#2, we custom extracted data from two STIS/FUV-MAMA exposures for the target GJ 3929. Now, we shall analyze its spectral time series. For this example, we are interested in the stellar fluxes coming from the Lyman-alpha line.

The first step of this analysis is to take advantage of the `TIME-TAG` mode, which assigns a timestamp to every count that hits the detector and allows for a temporal resolution down to that of the electronics! We do not need all of that. In fact, here we shall work with a coarser temporal resolution that is more relevant for the analysis of stellar flares.

We shall use the `stis_analysis.timetag_split()` function to break down a full exposure into sub-exposures. This could take a while because in the end we need to process many subexposures.

In [4]:
time_resolution = 60  # unit of seconds
prefix = './example_data/'
output_dir = './example_data/timeseries/'
datasets = ['ofkb08020', 'ofkb08030']

for dataset in datasets:
    stis_analysis.timetag_split(dataset, 
                                prefix, 
                                output_dir=output_dir, 
                                temporal_resolution=time_resolution, 
                                overwrite=True)

imset: 1, start: 0.026625, stop: 59.382945945945956, exposure time: 59.35632094594595
imset: 2, start: 59.382945945945956, stop: 118.7392668918919, exposure time: 59.356320945945946
imset: 3, start: 118.7392668918919, stop: 178.09558783783785, exposure time: 59.35632094594595
imset: 4, start: 178.09558783783785, stop: 237.4519087837838, exposure time: 59.35632094594595
imset: 5, start: 237.4519087837838, stop: 296.80822972972976, exposure time: 59.35632094594595
imset: 6, start: 296.80822972972976, stop: 356.16455067567574, exposure time: 59.35632094594598
imset: 7, start: 356.16455067567574, stop: 415.5208716216217, exposure time: 59.35632094594598
imset: 8, start: 415.5208716216217, stop: 474.8771925675677, exposure time: 59.35632094594598
imset: 9, start: 474.8771925675677, stop: 534.2335135135137, exposure time: 59.35632094594598
imset: 10, start: 534.2335135135137, stop: 593.5898344594597, exposure time: 59.35632094594598
imset: 11, start: 593.5898344594597, stop: 652.946155405405

The code above should have generated `*_ts_x1d.fits` files inside the output folder (`ts` stands for time-series). Inside these new files, there are many extensions that correspond to each subexposure.